In [1]:
!pip install stanza
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=ac982ba0e9f74946188985a430d7fdfeb611c86bb4de17690a478cba6235cd92
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


In [2]:
from google.colab import files
src = list(files.upload().values())[0]
open('utils.py','wb').write(src)
import utils

Saving utils.py to utils.py


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


glove path missing
Utils have been correctly loaded


In [3]:
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import save, load
import torch.nn as nn

from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset, InMemoryDataset
from torch_geometric.utils import scatter
from torch_geometric.data import download_url

from torch_geometric.nn import GCNConv, GATv2Conv, global_mean_pool
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from sklearn.metrics import classification_report

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
tweet_list_train = []
sentiment_list_train = []
tweet_list_test = []
sentiment_list_test = []
cv19_graph_data_train = utils.Dataset_from_sentences("train", "/content/processed_train/",
                                                     "/content/drive/MyDrive/GraphDataset/train/",
                                                     tweet_list_train, sentiment_list_train)
cv19_graph_data_test = utils.Dataset_from_sentences("test", "/content/processed_test/",
                                                    "/content/drive/MyDrive/GraphDataset/test/",
                                                    tweet_list_test, sentiment_list_test)
#strat_train = load("/content/drive/MyDrive/StratifiedSplit/strat_train.pt")
strat_val = load("/content/drive/MyDrive/StratifiedSplit/strat_val.pt")
print("Loaded training dataset:")
print(cv19_graph_data_train)
print("Loaded test dataset:")
print(cv19_graph_data_test)
#print("Loaded training and validation sets as result of stratified k-fold:")
#print(strat_train)
#print(strat_val)

['/content/processed_train//train.pt']
['/content/processed_test//test.pt']
Loaded training dataset:
Dataset_from_sentences(25948)
Loaded test dataset:
Dataset_from_sentences(6487)


In [8]:
class GAT(torch.nn.Module):
  """Graph Attention Network"""
  def __init__(self, dim_in, n_filters, dim_out, heads= 8):
    super().__init__()
    # dim_in is the number of node features, dim_h is the dimension
    # of the hidden layer, dim_out is the dimension of the output
    # feature vector
    self.first_linear = Linear(dim_in, dim_in)

    self.gat_list_1 = torch.nn.ModuleList([GATv2Conv(dim_in, dim_in // heads, heads = heads)
                                         for i in range(0, n_filters)])
    self.gat_list_2 = torch.nn.ModuleList([GATv2Conv(dim_in, dim_in // heads, heads = heads)
                                         for i in range(0, n_filters)])
    self.gat_list_3 = torch.nn.ModuleList([GATv2Conv(dim_in, dim_in // heads, heads = heads)
                                         for i in range(0, n_filters)])

    self.l_list = torch.nn.ModuleList([Linear(dim_in, 1)
                                      for i in range(0, n_filters)])
    self.classifier = Linear(n_filters, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=1e-3,
                                      weight_decay=5e-5)

  def forward(self, x, edge_index, batch, enable_log = False):
    # the parameters of the forward correspond to data.x and data.edge_index
    # where data is a Data object like those described above;
    if enable_log:
        utils.visualize_hidden_graph(x, edge_index)

    #h = self.first_linear(x)

    h_list = []
    for i, gat_l in enumerate(self.gat_list_1):
        #print(f"shape of initial tensor x: {x.shape}")
        h = x + gat_l(x, edge_index)
        #print(f"shape of the first residual: {h.shape}")
        h = h.tanh()
        h = h + self.gat_list_2[i](h, edge_index)
        #print(f"shape of the second residual: {h.shape}")
        h = h.tanh()
        h = h + self.gat_list_3[i](h, edge_index)
        #print(f"shape of the third residual: {h.shape}")
        if enable_log:
            print("h shape: " + str(h.shape))
            utils.visualize_hidden_graph(h, edge_index)
        h = global_mean_pool(h, batch)
        #print(f"shape of the tensor after global mean pool: {h.shape}")
        h = self.l_list[i](h)
        h_list.append(h)

    h_layers = torch.hstack(h_list)
    #print(h_layers.shape)
    h_layers = self.classifier(h_layers)
    return h_layers

In [9]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

In [9]:
gat_ca2 = load("/content/drive/MyDrive/Models/CosineAnnealingLR_2")
gat_ca3 = load("/content/drive/MyDrive/Models/CosineAnnealingLR_3")
gat_ca4 = load("/content/drive/MyDrive/Models/CosineAnnealingLR_4")
gat_oc = load("/content/drive/MyDrive/Models/OneCycleLR_1")
gat_pl = load("/content/drive/MyDrive/Models/PolynomialLR")
gat_pl2 = load("/content/drive/MyDrive/Models/PolynomialLR_2")
gat_st = load("/content/drive/MyDrive/Models/StepLR_0")
gat_st2 = load("/content/drive/MyDrive/Models/StepLR_1")

In [10]:
cv19_graph_data_test.to(device)

Dataset_from_sentences(6487)

In [10]:
strat_val.to(device)

Dataset_from_sentences(3244)

In [12]:
def process_test(model, test_set, batch_size = 256):
  loader_test =  DataLoader(test_set.data_list, batch_size=batch_size, shuffle=False)
  out_predictions = []
  for i, batch in enumerate(loader_test):
    out = model(batch.x, batch.edge_index, batch.batch)
    out_predictions.extend(out.detach().cpu())
    acc_test = accuracy(out.argmax(dim=1), batch.y)
  return out_predictions

def process_test_2(model, test_set):
  out = model(test_set.data_list.x, test_set.data_list.edge_index, test_set.data_list.batch)
  return out.detach().cpu()

def get_classification_report(model, test_set):
  out_list = process_test_2(model, test_set)
  out_idx = [vec.argmax().item() for vec in out_list]
  truth = [elem.item() for elem in cv19_graph_data_test.data_list.y.detach().cpu()]
  reprt = classification_report(truth, out_idx)
  return reprt

In [21]:
torch.cuda.empty_cache()
del out_list

In [37]:
print(process_test_2(gat_ca2, cv19_graph_data_test))

tensor([[ 33.3680,  27.0486,   6.0049,  10.5307,  -6.0365],
        [-32.6037, -11.3049,   2.2983,  15.2211,  35.1476],
        [ 10.8053,  18.9477,  13.5201,  22.8925,  14.1821],
        ...,
        [-12.4513,   8.0730,   3.7145,  16.6948,  19.0011],
        [ 26.8934,  23.2827,  13.6799,  17.1695,   6.2934],
        [ 21.7820,  24.0243,  18.7870,  18.4631,   3.0929]])


In [42]:
print(get_classification_report(gat_ca2, cv19_graph_data_test))

              precision    recall  f1-score   support

           0       0.55      0.57      0.56       829
           1       0.44      0.44      0.44      1482
           2       0.40      0.39      0.40       912
           3       0.47      0.48      0.47      1910
           4       0.64      0.63      0.64      1354

    accuracy                           0.50      6487
   macro avg       0.50      0.50      0.50      6487
weighted avg       0.50      0.50      0.50      6487



In [43]:
print(get_classification_report(gat_ca3, cv19_graph_data_test))

              precision    recall  f1-score   support

           0       0.56      0.56      0.56       829
           1       0.45      0.44      0.44      1482
           2       0.41      0.41      0.41       912
           3       0.46      0.48      0.47      1910
           4       0.64      0.61      0.63      1354

    accuracy                           0.50      6487
   macro avg       0.50      0.50      0.50      6487
weighted avg       0.50      0.50      0.50      6487



In [44]:
print(get_classification_report(gat_ca4, cv19_graph_data_test))

              precision    recall  f1-score   support

           0       0.57      0.58      0.57       829
           1       0.44      0.43      0.44      1482
           2       0.39      0.39      0.39       912
           3       0.47      0.49      0.48      1910
           4       0.65      0.64      0.64      1354

    accuracy                           0.50      6487
   macro avg       0.50      0.50      0.50      6487
weighted avg       0.50      0.50      0.50      6487



In [46]:
print(get_classification_report(gat_oc, cv19_graph_data_test))


              precision    recall  f1-score   support

           0       0.55      0.55      0.55       829
           1       0.44      0.44      0.44      1482
           2       0.38      0.37      0.37       912
           3       0.47      0.48      0.47      1910
           4       0.61      0.63      0.62      1354

    accuracy                           0.49      6487
   macro avg       0.49      0.49      0.49      6487
weighted avg       0.49      0.49      0.49      6487



In [47]:
print(get_classification_report(gat_pl, cv19_graph_data_test))


              precision    recall  f1-score   support

           0       0.56      0.57      0.57       829
           1       0.45      0.43      0.44      1482
           2       0.42      0.43      0.42       912
           3       0.47      0.49      0.48      1910
           4       0.63      0.61      0.62      1354

    accuracy                           0.50      6487
   macro avg       0.51      0.51      0.51      6487
weighted avg       0.50      0.50      0.50      6487



In [48]:
print(get_classification_report(gat_pl2, cv19_graph_data_test))


              precision    recall  f1-score   support

           0       0.60      0.59      0.59       829
           1       0.47      0.46      0.46      1482
           2       0.41      0.40      0.41       912
           3       0.49      0.50      0.49      1910
           4       0.65      0.65      0.65      1354

    accuracy                           0.52      6487
   macro avg       0.52      0.52      0.52      6487
weighted avg       0.52      0.52      0.52      6487



In [50]:
print(get_classification_report(gat_st, cv19_graph_data_test))


              precision    recall  f1-score   support

           0       0.56      0.57      0.56       829
           1       0.42      0.39      0.41      1482
           2       0.38      0.39      0.39       912
           3       0.47      0.49      0.48      1910
           4       0.64      0.62      0.63      1354

    accuracy                           0.49      6487
   macro avg       0.49      0.49      0.49      6487
weighted avg       0.49      0.49      0.49      6487



In [51]:
print(get_classification_report(gat_st2, cv19_graph_data_test))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58       829
           1       0.44      0.44      0.44      1482
           2       0.39      0.41      0.40       912
           3       0.48      0.47      0.47      1910
           4       0.64      0.63      0.64      1354

    accuracy                           0.50      6487
   macro avg       0.50      0.51      0.51      6487
weighted avg       0.50      0.50      0.50      6487



In [13]:
print(get_classification_report(gat_pl2, strat_val))

OutOfMemoryError: ignored